# **Normal Cantilever Beam:**

**Boundary Conditions:**   ```v[x = 0] = 0; dv/dx[x = 0] = 0```

---
Point Load
---

In [ ]:
import numpy as np

def newton_raphson_system(N, init_guess='uniform', dv_dx_at_firstPoint='FD2', tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method.

    Parameters:
        N: int - Number of grid spaces.
        tol: float - Tolerance for convergence.
        max_iter: int - Maximum number of iterations.

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    # Validate inputs
    valid_init_guesses = ['random', 'uniform', 'zeros']
    valid_dv_dx_methods = ['FD2', 'FD1', 'Direct']

    if init_guess not in valid_init_guesses:
        raise ValueError(f"Invalid initial guess '{init_guess}'. Must be one of {valid_init_guesses}.")
    if dv_dx_at_firstPoint not in valid_dv_dx_methods:
        raise ValueError(f"Invalid dv/dx method '{dv_dx_at_firstPoint}'. Must be one of {valid_dv_dx_methods}.")

    # Discretization
    L = 1.0
    dx = L / N
    x = np.linspace(0, L, N+1)
    EI =  2000       # Flexural rigidity (N·m^2)
    P  = -100.0      # Point load at the free end (N)
    w  =  0.5        # Relaxation factor

    # Initial guess
    if init_guess == 'random':
        v = np.random.uniform(0, 0.01 * np.sign(P), N+1)
        v[0] = 0
    elif init_guess == 'uniform':
        v = 0.0001 * np.sign(P) * x[:]
        v[0] = 0
    elif init_guess == 'zeros':
        v = np.zeros(N+1)
        v[0] = 0
    print()
    print(v)

    # Moment Equation
    M = np.zeros(N+1)
    for i in range(N+1):
        M[i] = -P*(L - x[i])

    # Residual Function
    def residual(v, M):
        """Compute the residual vector R(v)."""
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                if dv_dx_at_firstPoint == 'FD2':
                    R[i] = (-3*v_full[i]+4*v_full[i+1]-v_full[i+2])/(2*dx)
                elif dv_dx_at_firstPoint == 'FD1':
                    R[i] = (v_full[i+1]-v_full[i])/dx
                elif dv_dx_at_firstPoint == 'Direct':
                    R[i] = (2*v_full[i]-5*v_full[i+1]+4*v_full[i+2]-v_full[i+3])/dx**2 + (M[i]/EI)

            elif i == N:
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])/dx**2 + (M[i]/EI) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(3/2)
            else:
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])/dx**2 + (M[i]/EI) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v)."""
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                if dv_dx_at_firstPoint == 'FD2':
                    J[i, i], J[i, i+1], J[i, i+2] =  0, 4/(2*dx), -1/(2*dx)
                elif dv_dx_at_firstPoint == 'FD1':
                    J[i, i], J[i, i+1] = 0, 1/dx
                elif dv_dx_at_firstPoint == 'Direct':
                    J[i, i], J[i, i+1], J[i, i+2], J[i, i+3] =  0, -5/dx**2, 4/dx**2, -1/dx**2

            elif i == N:
                J[i, i-3] = -1/dx**2
                J[i, i-2] =  4/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * 1/(4*dx**2) * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (1)
                J[i, i-1] = -5/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * 1/(4*dx**2) * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) * (-4)
                J[i, i]   =  2/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * 1/(4*dx**2) * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (3)
            else:
                if i == 1:
                    J[i, i-1] = 0
                else:
                    J[i, i-1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i-1]-2*v_full[i+1])
                J[i, i]   = -2/dx**2
                J[i, i+1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i+1]-2*v_full[i-1])
        return J

    # Newton-Raphson iteration
    for iteration in range(max_iter):
        R = residual(v, M)
        print(np.linalg.norm(R))
        if np.linalg.norm(R) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        J = jacobian(v, M)
        try:
            # Attempt to solve the system
            delta_v = np.linalg.solve(J, -R)
        except np.linalg.LinAlgError:
            # If singular, use pseudo-inverse
            print(f"\nJacobian is singular for iteration {iteration+1}. Using pseudo-inverse.")
            delta_v = np.linalg.lstsq(J, -R, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist

        v += w*delta_v

        if np.linalg.norm(delta_v) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Full solution including boundaries
    v_full = np.zeros(N+1)
    v_full[:] = v
    v_full[0] = 0

    return x, v_full, L, P, EI


# Solve and visualize
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cufflinks as cf
import pandas as pd
'''
    Minimum value of 'N' can be 3.
    Don't take N > 350.  >>> Leads to Jacobian Matrix being singular for all three kinds of initial guess (random, uniform, zeros).
'''
N = 250
init_guess = input("Enter initial guess type (random, uniform, zeros): ")
dv_dx_at_firstPoint = input("Enter dv/dx at x = 0 (FD2, FD1, Direct): ")
x, v, L, P, EI = newton_raphson_system(N, init_guess, dv_dx_at_firstPoint)
print(v)
print()

# plt.plot(x, v, label="Newton-Raphson Solution")
# plt.plot(x, P*x**2/(6*EI) * (3*L - x), label="Euler-Bernoulli Solution")
# plt.xlabel("x")
# plt.ylabel("v(x)")
# plt.legend()
# plt.grid()
# plt.show()

# Create Pandas DataFrame
df = pd.DataFrame({'x': x, 'Newton-Raphson Solution (point load)': v, 'Euler-Bernoulli Solution (point load)': P*x**2/(6*EI) * (3*L - x)})

# Create interactive plot using Plotly
fig = df.iplot(kind='line', x='x', y=['Newton-Raphson Solution (point load)', 'Euler-Bernoulli Solution (point load)'], asFigure=True)
fig.show()


---
Comparison of Point Load v/s Distributed Load
---

In [ ]:
import numpy as np

def newton_raphson_system(N, init_guess, dv_dx_at_firstPoint, tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method.

    Parameters:
        N: int - Number of grid spaces.
        tol: float - Tolerance for convergence.
        max_iter: int - Maximum number of iterations.

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    # Validate inputs
    valid_init_guesses = ['random', 'uniform', 'zeros']
    valid_dv_dx_methods = ['FD2', 'FD1', 'Direct']

    if init_guess not in valid_init_guesses:
        raise ValueError(f"Invalid initial guess '{init_guess}'. Must be one of {valid_init_guesses}.")
    if dv_dx_at_firstPoint not in valid_dv_dx_methods:
        raise ValueError(f"Invalid dv/dx method '{dv_dx_at_firstPoint}'. Must be one of {valid_dv_dx_methods}.")

    # Discretization
    L = 1.0
    dx = L / N
    x = np.linspace(0, L, N+1)
    EI =  2000       # Flexural rigidity (N·m^2)
    P  = -100.0      # Point load at the free end (N)
    W  = -100.0      # Continuous Load (N/m)
    w  =  0.5        # Relaxation factor

    # Initial guess -- Continuous load
    if init_guess == 'random':
        v_cont = np.random.uniform(0, 0.01 * np.sign(W), N+1)
        v_cont[0] = 0
    elif init_guess == 'uniform':
        v_cont = 0.0001 * np.sign(W) * x[:]
        v_cont[0] = 0
    elif init_guess == 'zeros':
        v_cont = np.zeros(N+1)
        v_cont[0] = 0

    # Initial guess -- Point load
    if init_guess == 'random':
        v_point = np.random.uniform(0, 0.01 * np.sign(P), N+1)
        v_point[0] = 0
    elif init_guess == 'uniform':
        v_point = 0.0001 * np.sign(P) * x[:]
        v_point[0] = 0
    elif init_guess == 'zeros':
        v_point = np.zeros(N+1)
        v_point[0] = 0

    # Moment Equation
    M_cont = np.zeros(N+1)
    M_point = np.zeros(N+1)
    for i in range(N+1):
        M_cont[i] = W*L*x[i] - (W * (x[i]**2))/2 - (W * (L**2))/2
        M_point[i] = -P*(L - x[i])

    # Residual Function
    def residual(v, M):
        """Compute the residual vector R(v)."""
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                if dv_dx_at_firstPoint == 'FD2':
                    R[i] = (-3*v_full[i]+4*v_full[i+1]-v_full[i+2])/(2*dx)
                elif dv_dx_at_firstPoint == 'FD1':
                    R[i] = (v_full[i+1]-v_full[i])/dx
                elif dv_dx_at_firstPoint == 'Direct':
                    R[i] = (2*v_full[i]-5*v_full[i+1]+4*v_full[i+2]-v_full[i+3])/dx**2 + (M[i]/EI)
            elif i == N:
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])/dx**2 + (M[i]/EI) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(3/2)
            else:
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])/dx**2 + (M[i]/EI) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v)."""
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                if dv_dx_at_firstPoint == 'FD2':
                    J[i, i], J[i, i+1], J[i, i+2] =  0, 4/(2*dx), -1/(2*dx)
                elif dv_dx_at_firstPoint == 'FD1':
                    J[i, i], J[i, i+1] = 0, 1/dx
                elif dv_dx_at_firstPoint == 'Direct':
                    J[i, i], J[i, i+1], J[i, i+2], J[i, i+3] =  0, -5/dx**2, 4/dx**2, -1/dx**2

            elif i == N:
                J[i, i-3] = -1/dx**2
                J[i, i-2] =  4/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * 1/(4*dx**2) * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (1)
                J[i, i-1] = -5/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * 1/(4*dx**2) * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) * (-4)
                J[i, i]   =  2/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * 1/(4*dx**2) * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (3)
            else:
                if i == 1:
                    J[i, i-1] = 0
                else:
                  J[i, i-1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i-1]-2*v_full[i+1])
                J[i, i]   = -2/dx**2
                J[i, i+1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 +  1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i+1]-2*v_full[i-1])
        return J

    # Newton-Raphson iteration -- Continuous load
    for iteration in range(max_iter):
        R_cont = residual(v_cont, M_cont)
        print(np.linalg.norm(R_cont))
        if np.linalg.norm(R_cont) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        J_cont = jacobian(v_cont, M_cont)
        try:
            # Attempt to solve the system
            delta_v_cont = np.linalg.solve(J_cont, -R_cont)
        except np.linalg.LinAlgError:
            # If singular, use pseudo-inverse
            print(f"\nJacobian is singular for iteration {iteration+1}. Using pseudo-inverse.")
            delta_v_cont = np.linalg.lstsq(J_cont, -R_cont, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist(Using least square)

        v_cont += w*delta_v_cont

        if np.linalg.norm(delta_v_cont) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Newton-Raphson iteration -- Point load
    for iteration in range(max_iter):
        R_point = residual(v_point, M_point)
        print(np.linalg.norm(R_point))
        if np.linalg.norm(R_point) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        J_point = jacobian(v_point, M_point)
        try:
            # Attempt to solve the system
            delta_v_point = np.linalg.solve(J_point, -R_point)
        except np.linalg.LinAlgError:
            # If singular, use pseudo-inverse
            print(f"\nJacobian is singular for iteration {iteration+1}. Using pseudo-inverse.")
            delta_v_point = np.linalg.lstsq(J_point, -R_point, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist(Using least square)

        v_point += w*delta_v_point

        if np.linalg.norm(delta_v_point) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Full solution including boundaries
    v_full_cont = np.zeros(N+1)
    v_full_cont[:] = v_cont
    v_full_cont[0] = 0

    v_full_point = np.zeros(N+1)
    v_full_point[:] = v_point
    v_full_point[0] = 0

    return x, v_full_point, v_full_cont, L, P, W, EI


# Solve and visualize
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cufflinks as cf
import pandas as pd
'''
    Minimum value of 'N' can be 3.
    Don't take N > 350.  >>> Leads to Jacobian Matrix being singular for all three kinds of initial guess (random, uniform, zeros).
'''
N = 500
init_guess = input("Enter initial guess type (random, uniform, zeros): ")
dv_dx_at_firstPoint = input("Enter dv/dx at x = 0 (FD2, FD1, Direct): ")
x, v_point, v_cont, L, P, W, EI = newton_raphson_system(N, init_guess, dv_dx_at_firstPoint)
# print(v)
print()

# plt.plot(x, v, label="Newton-Raphson Solution")
# plt.plot(x, P*x**2/(6*EI) * (3*L - x), label="Euler-Bernoulli Solution")
# plt.xlabel("x")
# plt.ylabel("v(x)")
# plt.legend()
# plt.grid()
# plt.show()

# Create Pandas DataFrame
df = pd.DataFrame({'x': x, 'Newton-Raphson Solution (point load)': v_point, 'Newton-Raphson Solution (continuous load)': v_cont, 'Euler-Bernoulli Solution (point load)': P*x**2/(6*EI) * (3*L - x), 'Euler-Bernoulli Solution (continuous load)': -(W*L*x**3)/(6*EI) + (W*x**4)/(24*EI) + (W*(L**2)*x**2)/(4*EI)})

# Create interactive plot using Plotly
fig = df.iplot(kind='line', x='x', y=['Newton-Raphson Solution (point load)', 'Newton-Raphson Solution (continuous load)', 'Euler-Bernoulli Solution (point load)', 'Euler-Bernoulli Solution (continuous load)'], asFigure=True)
fig.show()


---
Implementation of Point Load and Uniform Continuous Load
---

In [ ]:
import numpy as np

def newton_raphson_system(N, init_guess, dv_dx_at_firstPoint, tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method.

    Parameters:
        N: int - Number of grid spaces.
        tol: float - Tolerance for convergence.
        max_iter: int - Maximum number of iterations.

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    # Validate inputs
    valid_init_guesses = ['random', 'uniform', 'zeros']
    valid_dv_dx_methods = ['FD2', 'FD1', 'Direct']

    if init_guess not in valid_init_guesses:
        raise ValueError(f"Invalid initial guess '{init_guess}'. Must be one of {valid_init_guesses}.")
    if dv_dx_at_firstPoint not in valid_dv_dx_methods:
        raise ValueError(f"Invalid dv/dx method '{dv_dx_at_firstPoint}'. Must be one of {valid_dv_dx_methods}.")

    # Discretization
    L = 1.0
    dx = L / N
    x = np.linspace(0, L, N+1)
    EI =  2000       # Flexural rigidity (N·m^2)
    P  = -100.0      # Point load at the free end (N)
    W  = -100.0      # Continuous Load (N/m)
    w  =  0.5        # Relaxation factor

    # Initial guess
    v = np.zeros(N+1)          # Good guess. Achieves Middle Ground.
    v[0] = 0

    # Getting the Point Loads
    def get_point_loads():
        n = int(input("Enter the number of point loads: "))
        point_loads = []
        for i in range(n):
            pos, mag = map(float, input(f"Enter position and magnitude of point load {i+1} (space-separated): ").split())
            point_loads.append((pos, mag))
        return point_loads

    # Getting the Uniform Continuous Loads
    def get_continuous_loads():
        m = int(input("Enter the number of continuous loads: "))
        continuous_loads = []
        for i in range(m):
            start, end, intensity = map(float, input(f"Enter start position, end position, and intensity of continuous load {i+1} (space-separated): ").split())
            continuous_loads.append((start, end, intensity))
        return continuous_loads

    # Generating the Moment Equation
    def moment_equation(L, point_loads, continuous_loads, N):
        x_vals = np.linspace(0, L, N+1)
        moment_values = np.zeros(N+1)

        for i, x in enumerate(x_vals):
            M = 0
            for pos, mag in point_loads:
                if not (0 <= pos <= L):
                    raise ValueError(f"Point load at {pos} exceeds beam length {L}.")
                if x <= pos:
                    M += mag * (x - pos)

            for start, end, intensity in continuous_loads:
                if not (0 <= start <= end <= L):
                    raise ValueError(f"Continuous load from {start} to {end} exceeds beam length {L}.")

                if x <= start:
                    M += -intensity * (end - start) * (end + start) / 2 + intensity * (end - start) * x
                elif start < x < end:
                    M += -intensity * (end - start) * (end + start) / 2 + intensity * (end - start) * x - intensity * (x - start)**2 / 2
                else:
                    M += -intensity * (end - start) * (end + start) / 2 + intensity * (end - start) * x - intensity * (end - start) * (x - (start + end) / 2)

            moment_values[i] = M

        return moment_values

    # Residual Function
    def residual(v, M):
        """Compute the residual vector R(v)."""
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                if dv_dx_at_firstPoint == 'FD2':
                    R[i] = (-3*v_full[i]+4*v_full[i+1]-v_full[i+2])/(2*dx)
                elif dv_dx_at_firstPoint == 'FD1':
                    R[i] = (v_full[i+1]-v_full[i])/dx
                elif dv_dx_at_firstPoint == 'Direct':
                    R[i] = (2*v_full[i]-5*v_full[i+1]+4*v_full[i+2]-v_full[i+3])/dx**2 + (M[i]/EI)
            elif i == N:
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])/dx**2 + (M[i]/EI) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(3/2)
            else:
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])/dx**2 + (M[i]/EI) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v)."""
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition dv/dx[x=0] = 0
                if dv_dx_at_firstPoint == 'FD2':
                    J[i, i], J[i, i+1], J[i, i+2] =  0, 4/(2*dx), -1/(2*dx)
                elif dv_dx_at_firstPoint == 'FD1':
                    J[i, i], J[i, i+1] = 0, 1/dx
                elif dv_dx_at_firstPoint == 'Direct':
                    J[i, i], J[i, i+1], J[i, i+2], J[i, i+3] =  0, -5/dx**2, 4/dx**2, -1/dx**2

            elif i == N:
                J[i, i-3] = -1/dx**2
                J[i, i-2] =  4/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * 1/(4*dx**2) * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (1)
                J[i, i-1] = -5/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * 1/(4*dx**2) * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) * (-4)
                J[i, i]   =  2/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (3*v_full[i]-4*v_full[i-1]+v_full[i-2])**2)**(1/2) * 1/(4*dx**2) * 2 *(3*v_full[i]-4*v_full[i-1]+v_full[i-2]) *  (3)
            else:
                if i == 1:
                    J[i, i-1] = 0
                else:
                  J[i, i-1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i-1]-2*v_full[i+1])
                J[i, i]   = -2/dx**2
                J[i, i+1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 +  1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i+1]-2*v_full[i-1])
        return J

    point_loads = get_point_loads()
    continuous_loads = get_continuous_loads()
    moment_values = moment_equation(L, point_loads, continuous_loads, N)
    print()

    # Newton-Raphson iteration
    for iteration in range(max_iter):
        R = residual(v, moment_values)
        print(f"Residual = {np.linalg.norm(R)}")
        if np.linalg.norm(R) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        J = jacobian(v, moment_values)
        try:
            # Attempt to solve the system
            delta_v = np.linalg.solve(J, -R)
        except np.linalg.LinAlgError:
            # If singular, use pseudo-inverse
            print(f"Jacobian is singular for iteration {iteration+1}. Using pseudo-inverse.\n")
            delta_v = np.linalg.lstsq(J, -R, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist(Using least square)

        v += w*delta_v

        if np.linalg.norm(delta_v) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Full solution including boundaries
    v_full = np.zeros(N+1)
    v_full[:] = v
    v_full[0] = 0

    return x, v_full, L, P, W, EI


# Solve and visualize
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cufflinks as cf
import pandas as pd
'''
    Minimum value of 'N' can be 3.
    Don't take N > 350.  >>> Leads to Jacobian Matrix being singular for all three kinds of initial guess (random, uniform, zeros).
'''
N = 500
init_guess = input("Enter initial guess type (random, uniform, zeros): ")
dv_dx_at_firstPoint = input("Enter dv/dx at x = 0 (FD2, FD1, Direct): ")
x, v, L, P, W, EI = newton_raphson_system(N, init_guess, dv_dx_at_firstPoint)
# print(v)
print()

# plt.plot(x, v, label="Newton-Raphson Solution")
# plt.plot(x, P*x**2/(6*EI) * (3*L - x), label="Euler-Bernoulli Solution")
# plt.xlabel("x")
# plt.ylabel("v(x)")
# plt.legend()
# plt.grid()
# plt.show()

# Create Pandas DataFrame
df = pd.DataFrame({'x': x, 'Newton-Raphson Solution': v, 'Euler-Bernoulli Solution (point load)': P*x**2/(6*EI) * (3*L - x), 'Euler-Bernoulli Solution (continuous load)': -(W*L*x**3)/(6*EI) + (W*x**4)/(24*EI) + (W*(L**2)*x**2)/(4*EI)})

# Create interactive plot using Plotly
fig = df.iplot(kind='line', x='x', y=['Newton-Raphson Solution', 'Euler-Bernoulli Solution (point load)', 'Euler-Bernoulli Solution (continuous load)'], asFigure=True)
fig.show()


# **Simply Supported Beam:**

**Boundary Conditions:**   `v[x = 0] = 0; v[x = L] = 0; dv/dx[x = 0] = 0`

Explanation:

*   `v[x = 0] = 0` means the beam cannot deflect at the left support because it is pinned.
*   `v[x = L] = 0` means the beam cannot deflect at the right support because it is pinned.

*   `d2v/dx2[x = 0] = 0` means the beam is free to rotate and does not experience torque at the left support.

*   `d2v/dx2[x = L] = 0` means the beam does not experience bending moments at the right support.

---
Point Load
---

In [ ]:
import numpy as np

def newton_raphson_system(N, init_guess='uniform', tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method.

    Parameters:
        N: int - Number of grid spaces.
        tol: float - Tolerance for convergence.
        max_iter: int - Maximum number of iterations.

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    # Validate inputs
    valid_init_guesses = ['random', 'uniform', 'zeros']

    if init_guess not in valid_init_guesses:
        raise ValueError(f"Invalid initial guess '{init_guess}'. Must be one of {valid_init_guesses}.")

    # Discretization
    L = 1.0
    dx = L / N
    x = np.linspace(0, L, N+1)
    EI =  2000       # Flexural rigidity (N·m^2)
    P  = -100.0      # Point load at the middle (N)
    w  =  0.5        # Relaxation factor

    # Initial guess
    if init_guess == 'random':
        v = np.random.uniform(0, 0.01 * np.sign(P), N+1)
        v[0] = 0
        v[-1] = 0
    elif init_guess == 'uniform':
        v = 0.0001 * np.sign(P) * x[:]
        v[0] = 0
        v[-1] = 0
    elif init_guess == 'zeros':
        v = np.zeros(N+1)
        v[0] = 0
        v[-1] = 0
    print()
    print(v)

    # Moment Equation
    M = np.zeros(N+1)
    for i in range(N+1):
        if x[i] <= L/2:
            M[i] = (P/2) * x[i]
        else:
            M[i] = (P/2) * (L - x[i])

    # Residual Function
    def residual(v, M):
        """Compute the residual vector R(v)."""
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0
        v_full[-1] = 0          # Boundary condition v(L) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition d2v/dx2[x=0] = 0
                R[i] = (2*v_full[i]-5*v_full[i+1]+4*v_full[i+2]-v_full[i+3])/dx**2
            elif i == N:
                # Boundary condition d2v/dx2[x=L] = 0
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])/dx**2
            else:
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])/dx**2 + (M[i]/EI) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v)."""
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0
        v_full[-1] = 0          # Boundary condition v(L) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition d2v/dx2[x=0] = 0
                J[i, i], J[i, i+1], J[i, i+2], J[i, i+3] =  0, -5/dx**2, 4/dx**2, -1/dx**2
            elif i == N:
                # Boundary condition d2v/dx2[x=L] = 0
                J[i, i], J[i, i-1], J[i, i-2], J[i, i-3] =  0, -5/dx**2, 4/dx**2, -1/dx**2
            else:
                if i == 1:
                    J[i, i-1] = 0
                else:
                    J[i, i-1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i-1]-2*v_full[i+1])
                J[i, i]   = -2/dx**2
                J[i, i+1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i+1]-2*v_full[i-1])
        return J

    # Newton-Raphson iteration
    for iteration in range(max_iter):
        R = residual(v, M)
        print(np.linalg.norm(R))
        if np.linalg.norm(R) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        J = jacobian(v, M)
        try:
            # Attempt to solve the system
            delta_v = np.linalg.solve(J, -R)
        except np.linalg.LinAlgError:
            # If singular, use pseudo-inverse
            print(f"\nJacobian is singular for iteration {iteration+1}. Using pseudo-inverse.")
            delta_v = np.linalg.lstsq(J, -R, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist

        v += w*delta_v

        if np.linalg.norm(delta_v) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Full solution including boundaries
    v_full = np.zeros(N+1)
    v_full[:] = v
    v_full[0] = 0
    v_full[-1] = 0

    return x, v_full, L, P, EI


# Solve and visualize
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cufflinks as cf
import pandas as pd
'''
    Minimum value of 'N' can be 3.
    Don't take N > 350.  >>> Leads to Jacobian Matrix being singular for all three kinds of initial guess (random, uniform, zeros).
'''
N = 250
init_guess = input("Enter initial guess type (random, uniform, zeros): ")
x, v, L, P, EI = newton_raphson_system(N, init_guess)
print(v)
print()

# plt.plot(x, v, label="Newton-Raphson Solution")
# plt.plot(x, np.where(x <= L/2, (P * x / (12 * EI)) * ((3/4) * L**2 - x**2), (P * (L - x) / (12 * EI)) * (-x**2 + 2*L*x - (1/4) * L**2)), label="Euler-Bernoulli Solution")
# plt.xlabel("x")
# plt.ylabel("v(x)")
# plt.legend()
# plt.grid()
# plt.show()

# Create Pandas DataFrame
df = pd.DataFrame({'x': x, 'Newton-Raphson Solution (Point Load)': v, 'Euler-Bernoulli Solution (Point Load)': np.where(x <= L/2, (P * x / (12 * EI)) * ((3/4) * L**2 - x**2), (P * (L - x) / (12 * EI)) * (-x**2 + 2*L*x - (1/4) * L**2))})

# Create interactive plot using Plotly
fig = df.iplot(kind='line', x='x', y=['Newton-Raphson Solution (Point Load)', 'Euler-Bernoulli Solution (Point Load)'], asFigure=True)
fig.show()


---
Comparison of Point Load v/s Distributed Load
---

In [ ]:
import numpy as np

def newton_raphson_system(N, init_guess='uniform', tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method.

    Parameters:
        N: int - Number of grid spaces.
        tol: float - Tolerance for convergence.
        max_iter: int - Maximum number of iterations.

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    # Validate inputs
    valid_init_guesses = ['random', 'uniform', 'zeros']

    if init_guess not in valid_init_guesses:
        raise ValueError(f"Invalid initial guess '{init_guess}'. Must be one of {valid_init_guesses}.")

    # Discretization
    L = 1.0
    dx = L / N
    x = np.linspace(0, L, N+1)
    EI =  2000       # Flexural rigidity (N·m^2)
    P  = -100.0      # Point load at the middle (N)
    W  = -100.0      # Continuous Load (N/m)
    w  =  0.5        # Relaxation factor

    # Initial guess -- Continuous load
    if init_guess == 'random':
        v_cont = np.random.uniform(0, 0.01 * np.sign(W), N+1)
        v_cont[0] = 0
        v_cont[-1] = 0
    elif init_guess == 'uniform':
        v_cont = 0.0001 * np.sign(W) * x[:]
        v_cont[0] = 0
        v_cont[-1] = 0
    elif init_guess == 'zeros':
        v_cont = np.zeros(N+1)
        v_cont[0] = 0
        v_cont[-1] = 0

    # Initial guess -- Point load
    if init_guess == 'random':
        v_point = np.random.uniform(0, 0.01 * np.sign(P), N+1)
        v_point[0] = 0
        v_point[-1] = 0
    elif init_guess == 'uniform':
        v_point = 0.0001 * np.sign(P) * x[:]
        v_point[0] = 0
        v_point[-1] = 0
    elif init_guess == 'zeros':
        v_point = np.zeros(N+1)
        v_point[0] = 0
        v_point[-1] = 0

    # Moment Equation
    M_cont = np.zeros(N+1)
    M_point = np.zeros(N+1)
    for i in range(N+1):
        # Continuous Load
        M_cont[i] = 1/2 * W * x[i] * (L - x[i])

        # Point Load
        if x[i] <= L/2:
            M_point[i] = (P/2) * x[i]
        else:
            M_point[i] = (P/2) * (L - x[i])

    # Residual Function
    def residual(v, M):
        """Compute the residual vector R(v)."""
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0
        v_full[-1] = 0          # Boundary condition v(L) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition d2v/dx2[x=0] = 0
                R[i] = (2*v_full[i]-5*v_full[i+1]+4*v_full[i+2]-v_full[i+3])/dx**2
            elif i == N:
                # Boundary condition d2v/dx2[x=L] = 0
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])/dx**2
            else:
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])/dx**2 + (M[i]/EI) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v)."""
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0
        v_full[-1] = 0          # Boundary condition v(L) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition d2v/dx2[x=0] = 0
                J[i, i], J[i, i+1], J[i, i+2], J[i, i+3] =  0, -5/dx**2, 4/dx**2, -1/dx**2
            elif i == N:
                # Boundary condition d2v/dx2[x=L] = 0
                J[i, i], J[i, i-1], J[i, i-2], J[i, i-3] =  0, -5/dx**2, 4/dx**2, -1/dx**2
            else:
                if i == 1:
                    J[i, i-1] = 0
                else:
                    J[i, i-1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i-1]-2*v_full[i+1])
                J[i, i]   = -2/dx**2
                J[i, i+1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i+1]-2*v_full[i-1])
        return J

    # Newton-Raphson iteration -- Continuous load
    for iteration in range(max_iter):
        R_cont = residual(v_cont, M_cont)
        print(np.linalg.norm(R_cont))
        if np.linalg.norm(R_cont) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        J_cont = jacobian(v_cont, M_cont)
        try:
            # Attempt to solve the system
            delta_v_cont = np.linalg.solve(J_cont, -R_cont)
        except np.linalg.LinAlgError:
            # If singular, use pseudo-inverse
            print(f"\nJacobian is singular for iteration {iteration+1}. Using pseudo-inverse.")
            delta_v_cont = np.linalg.lstsq(J_cont, -R_cont, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist(Using least square)

        v_cont += w*delta_v_cont

        if np.linalg.norm(delta_v_cont) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Newton-Raphson iteration -- Point load
    for iteration in range(max_iter):
        R_point = residual(v_point, M_point)
        print(np.linalg.norm(R_point))
        if np.linalg.norm(R_point) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        J_point = jacobian(v_point, M_point)
        try:
            # Attempt to solve the system
            delta_v_point = np.linalg.solve(J_point, -R_point)
        except np.linalg.LinAlgError:
            # If singular, use pseudo-inverse
            print(f"\nJacobian is singular for iteration {iteration+1}. Using pseudo-inverse.")
            delta_v_point = np.linalg.lstsq(J_point, -R_point, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist(Using least square)

        v_point += w*delta_v_point

        if np.linalg.norm(delta_v_point) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Full solution including boundaries
    v_full_cont = np.zeros(N+1)
    v_full_cont[:] = v_cont
    v_full_cont[0] = 0
    v_full_cont[-1] = 0

    v_full_point = np.zeros(N+1)
    v_full_point[:] = v_point
    v_full_point[0] = 0
    v_full_point[-1] = 0

    return x, v_full_point, v_full_cont, L, P, W, EI


# Solve and visualize
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cufflinks as cf
import pandas as pd
'''
    Minimum value of 'N' can be 3.
    Don't take N > 350.  >>> Leads to Jacobian Matrix being singular for all three kinds of initial guess (random, uniform, zeros).
'''
N = 250
init_guess = input("Enter initial guess type (random, uniform, zeros): ")
x, v_point, v_cont, L, P, W, EI = newton_raphson_system(N, init_guess)
# print(v)
print()

# Create Pandas DataFrame
df = pd.DataFrame({'x': x, 'Newton-Raphson Solution (Point load)': v_point, 'Newton-Raphson Solution (Continuous load)': v_cont, 'Euler-Bernoulli Solution (Point load)': np.where(x <= L/2, (P * x / (12 * EI)) * ((3/4) * L**2 - x**2), (P * (L - x) / (12 * EI)) * (-x**2 + 2*L*x - (1/4) * L**2)), \
                   'Euler-Bernoulli Solution (Continuous load)': W*x/(24*EI) * (L**3 - 2*L*x**2 + x**3)})

# Create interactive plot using Plotly
fig = df.iplot(kind='line', x='x', y=['Newton-Raphson Solution (Point load)', 'Newton-Raphson Solution (Continuous load)', 'Euler-Bernoulli Solution (Point load)', 'Euler-Bernoulli Solution (Continuous load)'], asFigure=True)
fig.show()


---
Implementation of Point Load and Uniform Continuous Load
---

In [ ]:
import numpy as np

def newton_raphson_system(N, init_guess, tol=1e-6, max_iter=1000):
    """
    Solve the nonlinear system using Newton-Raphson method.

    Parameters:
        N: int - Number of grid spaces.
        tol: float - Tolerance for convergence.
        max_iter: int - Maximum number of iterations.

    Returns:
        x: array - Discretized x values.
        v: array - Solution v(x).
    """

    # Validate inputs
    valid_init_guesses = ['random', 'uniform', 'zeros']

    if init_guess not in valid_init_guesses:
        raise ValueError(f"Invalid initial guess '{init_guess}'. Must be one of {valid_init_guesses}.")

    # Discretization
    L = 1.0
    dx = L / N
    x = np.linspace(0, L, N+1)
    EI =  2000       # Flexural rigidity (N·m^2)
    P  = -100.0      # Point load at the middle (N)
    W  = -100.0      # Continuous Load (N/m)
    w  =  0.5        # Relaxation factor

    # Initial guess
    v = np.zeros(N+1)          # Good guess. Achieves Middle Ground.
    v[0] = 0
    v[-1] = 0

    # Getting the Point Loads
    def get_point_loads():
        n = int(input("Enter the number of point loads: "))
        point_loads = []
        for i in range(n):
            pos, mag = map(float, input(f"Enter position and magnitude of point load {i+1} (space-separated): ").split())
            point_loads.append((pos, mag))
        return point_loads

    # Getting the Uniform Continuous Loads
    def get_continuous_loads():
        m = int(input("Enter the number of continuous loads: "))
        continuous_loads = []
        for i in range(m):
            start, end, intensity = map(float, input(f"Enter start position, end position, and intensity of continuous load {i+1} (space-separated): ").split())
            continuous_loads.append((start, end, intensity))
        return continuous_loads

    # Generating the Moment Equation
    def moment_equation(L, point_loads, continuous_loads, N):
        x_vals = np.linspace(0, L, N+1)
        moment_values = np.zeros(N+1)

        for i, x in enumerate(x_vals):
            M = 0
            for pos, mag in point_loads:
                if not (0 <= pos <= L):
                    raise ValueError(f"Point load at {pos} exceeds beam length {L}.")
                if x <= pos:
                    M += mag * (L - pos)/L * x
                else:
                    M += mag * pos * (1 - x/L)

            for start, end, intensity in continuous_loads:
                if not (0 <= start <= end <= L):
                    raise ValueError(f"Continuous load from {start} to {end} exceeds beam length {L}.")

                if x <= start:
                    M += intensity * (end - start) * (1 - (end + start) / (2 * L)) * x
                elif start < x < end:
                    M += intensity * (end - start) * (1 - (end + start) / (2 * L)) * x - intensity * (x - start) * (x - start) / 2
                else:
                    M += intensity * (end - start) * (end + start) / (2 * L) * (L - x)

            moment_values[i] = M

        return moment_values

    # Residual Function
    def residual(v, M):
        """Compute the residual vector R(v)."""
        R = np.zeros(N+1)
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0
        v_full[-1] = 0          # Boundary condition v(L) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition d2v/dx2[x=0] = 0
                R[i] = (2*v_full[i]-5*v_full[i+1]+4*v_full[i+2]-v_full[i+3])/dx**2
            elif i == N:
                # Boundary condition d2v/dx2[x=L] = 0
                R[i] = (2*v_full[i]-5*v_full[i-1]+4*v_full[i-2]-v_full[i-3])/dx**2
            else:
                R[i] = (v_full[i+1]-2*v_full[i]+v_full[i-1])/dx**2 + (M[i]/EI) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(3/2)
        return R

    # Jacobian Function
    def jacobian(v, M):
        """Compute the Jacobian matrix J(v)."""
        J = np.zeros((N+1, N+1))
        v_full = np.zeros(N+1)
        v_full[:] = v
        v_full[0] = 0           # Boundary condition v(0) = 0
        v_full[-1] = 0          # Boundary condition v(L) = 0

        for i in range(0, N+1):
            if i == 0:
                # Boundary condition d2v/dx2[x=0] = 0
                J[i, i], J[i, i+1], J[i, i+2], J[i, i+3] =  0, -5/dx**2, 4/dx**2, -1/dx**2
            elif i == N:
                # Boundary condition d2v/dx2[x=L] = 0
                J[i, i], J[i, i-1], J[i, i-2], J[i, i-3] =  0, -5/dx**2, 4/dx**2, -1/dx**2
            else:
                if i == 1:
                    J[i, i-1] = 0
                else:
                    J[i, i-1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i-1]-2*v_full[i+1])
                J[i, i]   = -2/dx**2
                J[i, i+1] =  1/dx**2 + (M[i]/EI) * (3/2) * (1 + 1/(4*dx**2) * (v_full[i+1]-v_full[i-1])**2)**(1/2) * 1/(4*dx**2) * (2*v_full[i+1]-2*v_full[i-1])
        return J

    point_loads = get_point_loads()
    continuous_loads = get_continuous_loads()
    moment_values = moment_equation(L, point_loads, continuous_loads, N)
    print()

    # Newton-Raphson iteration
    for iteration in range(max_iter):
        R = residual(v, moment_values)
        print(f"Residual = {np.linalg.norm(R)}")
        if np.linalg.norm(R) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break

        J = jacobian(v, moment_values)
        try:
            # Attempt to solve the system
            delta_v = np.linalg.solve(J, -R)
        except np.linalg.LinAlgError:
            # If singular, use pseudo-inverse
            print(f"Jacobian is singular for iteration {iteration+1}. Using pseudo-inverse.\n")
            delta_v = np.linalg.lstsq(J, -R, rcond=None)[0]    # Uses pseudo-inverse as the inverse does not exist(Using least square)

        v += w*delta_v

        if np.linalg.norm(delta_v) < tol:
            print(f"\nConverged in {iteration+1} iterations.\n")
            break
    else:
        raise ValueError(f"Newton-Raphson did not converge within the maximum number of iterations = {iteration+1}.")

    # Full solution including boundaries
    v_full = np.zeros(N+1)
    v_full[:] = v
    v_full[0] = 0
    v_full[-1] = 0

    return x, v_full, L, P, W, EI


# Solve and visualize
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cufflinks as cf
import pandas as pd
'''
    Minimum value of 'N' can be 3.
    Don't take N > 350.  >>> Leads to Jacobian Matrix being singular for all three kinds of initial guess (random, uniform, zeros).
'''
N = 250
init_guess = input("Enter initial guess type (random, uniform, zeros): ")
x, v, L, P, W, EI = newton_raphson_system(N, init_guess)
# print(v)
print()

# Create Pandas DataFrame
df = pd.DataFrame({'x': x, 'Newton-Raphson Solution': v, 'Euler-Bernoulli Solution (Point load)': np.where(x <= L/2, (P * x / (12 * EI)) * ((3/4) * L**2 - x**2), (P * (L - x) / (12 * EI)) * (-x**2 + 2*L*x - (1/4) * L**2)), \
                   'Euler-Bernoulli Solution (Continuous load)': W*x/(24*EI) * (L**3 - 2*L*x**2 + x**3)})

# Create interactive plot using Plotly
fig = df.iplot(kind='line', x='x', y=['Newton-Raphson Solution', 'Euler-Bernoulli Solution (Point load)', 'Euler-Bernoulli Solution (Continuous load)'], asFigure=True)
fig.show()
